#ex00

In [2]:
import pandas as pd
import sqlite3

Создадим подключение к базе данных с помощью библиотеки sqlite3

In [18]:
col_names = ['datetime', 'user']
connect = sqlite3.connect('../data/checking-logs.sqlite')

Получим схему просмотров страниц таблицы с помощью pd.io.sql.read_sql и запроса

Загрузить данные из SQL базы можно с помощью функции pd.read_SQL. Она автоматически преобразует столбцы SQL в столбцы DataFrame.

Команда PRAGMA в SQLite для получения метаданных.PRAGMA table_info(tableName) возвращает одну строку для каждого столбца в таблице.

http://python-3.ru/page/sqlite-metadata

In [10]:
query = 'PRAGMA table_info(pageviews);'
pd.io.sql.read_sql(query, connect)

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,uid,TEXT,0,None,0
2,2,datetime,TIMESTAMP,0,None,0


Получим первые 10 строк просмотров страниц таблицы, чтобы проверить, как выглядит таблица

https://pythonworld.ru/obrabotka-dannyx/pandas-cookbook-8.html

In [11]:
query = 'SELECT * FROM pageviews LIMIT 10;'
pd.io.sql.read_sql(query, connect)

,index,uid,datetime
0,0,admin_1,2020-04-17 12:01:08.463179
1,1,admin_1,2020-04-17 12:01:23.743946
2,2,admin_3,2020-04-17 12:17:39.287778
3,3,admin_3,2020-04-17 12:17:40.001768
4,4,admin_1,2020-04-17 12:27:30.646665
5,5,admin_1,2020-04-17 12:35:44.884757
6,6,admin_1,2020-04-17 12:35:52.735016
7,7,admin_3,2020-04-17 12:36:21.401412
8,8,admin_3,2020-04-17 12:36:22.023355
9,9,admin_1,2020-04-17 13:55:19.129243


Получим подтаблицу, учитывая все условия задания

https://pythonru.com/biblioteki/chtenie-i-zapis-dannyh-excel-json-sql-mongodb-pd-8

https://www.site-do.ru/db/sql4.php

SELECT какие столбцы выбираем

FROM откуда

WHERE uid like "user_%" такие что содержат фрагмент user_ и дальше что угодно

ORDER BY uid asc сортировка по возрастанию

https://runebook.dev/ru/docs/pandas/reference/api/pandas.read_sql

parse_dates - Список названий колонок для разбора в качестве дат

In [12]:
query = '''
SELECT uid, datetime
FROM pageviews
WHERE uid like "user_%"
ORDER BY uid asc
'''
pageviews = pd.io.sql.read_sql(query, connect, index_col='datetime', parse_dates=['datetime'])
pageviews

,uid
datetime,
2020-04-26 21:53:59.624136,user_1
2020-04-26 22:06:19.478143,user_1
2020-04-26 22:12:09.614497,user_1
2020-04-30 19:29:01.831635,user_1
2020-05-05 20:26:32.894852,user_1
...,...
2020-04-29 16:51:21.877630,user_30
2020-05-09 20:30:47.034282,user_30
2020-05-22 11:30:18.368990,user_5


Закроем соединение с базой данных

In [13]:
connect.close()

#ex01

Создадим подключение к базе данных с помощью библиотеки sqlite3

Получим схему просмотров страниц таблицы

Выберем строки из таблицы по условиям:

∗ status = ’ready’

∗ numTrials = 1

∗ labnames should be from the list: ’laba04’, ’laba04s’, ’laba05’, ’laba06’, ’laba06s’, ’project1’

◦ сохраним в фрейме данных контрольные элементы со столбцом cnt

In [19]:
request = '''
SELECT COUNT(*)
FROM pageviews
WHERE pageviews.uid IN
    (SELECT checker.uid
    FROM checker
    WHERE status='ready'
    AND numTrials=1
    AND labname in ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s', 'project1')
    )
'''
checkers = pd.io.sql.read_sql(request, connect)
checkers.columns = ['cnt']
checkers

,cnt
0,985


Закроем соединение с базой данных

In [ ]:
connect.close()

#ex02

In [ ]:
import pandas as pd
import sqlite3

In [ ]:
from google.colab import drive  # если вы выполняете код из среды Google Colab, нужно подключить свой гугл-диск,
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Создадим подключение к базе данных с помощью библиотеки sqlite3

In [ ]:
connect = sqlite3.connect('/content/drive/MyDrive/Pyton/checking-logs.sqlite.sqlite copy')

Создадим новую таблицу datamart в базе данных, объединив просмотры страниц таблиц и проверку с использованием только одного запроса

◦ в таблице должны быть следующие столбцы: uid, labname, first_commit_ts,
first_view_ts

фильтры те же:

∗ status = ’ready’

∗ numTrials = 1

∗ labnames should be from the list: ’laba04’, ’laba04s’, ’laba05’, ’laba06’, ’laba06s’, ’project1’

◦ таблица должна содержать только пользователей (uid с user_*), а не администраторов

◦ first_commit_ts и first_view_ts должны быть проанализированы как datetime64[ns]

In [20]:
request = '''
SELECT checker.uid,
       checker.labname,
       checker.timestamp AS first_commit_ts,
       pageviews.datetime AS first_view_ts
FROM checker 
LEFT JOIN pageviews ON checker.uid=pageviews.uid
WHERE status='ready'
AND numTrials=1
AND labname IN ('laba04', 'laba04s', 'laba05', 'laba06', 'laba06s', 'project1')
AND checker.uid LIKE 'user_%'
AND (pageviews.datetime = (SELECT MIN(pageviews.datetime)
                           FROM pageviews
                           WHERE uid=checker.uid)
     OR pageviews.datetime IS NULL)
'''
datamart = pd.io.sql.read_sql(request, connect, parse_dates=[
                              'first_commit_ts', 'first_view_ts'])
datamart

,uid,labname,first_commit_ts,first_view_ts
0,user_4,project1,2020-04-17 05:19:02.744528,NaT
1,user_4,laba04,2020-04-17 11:33:17.366400,NaT
2,user_4,laba04s,2020-04-17 11:48:41.992466,NaT
3,user_17,project1,2020-04-18 07:56:45.408648,2020-04-18 10:56:55.833899
4,user_30,laba04,2020-04-18 13:36:53.971502,2020-04-17 22:46:26.785035
...,...,...,...,...
135,user_23,laba06,2020-05-21 08:34:10.517205,NaT
136,user_19,laba06s,2020-05-21 13:27:06.705881,2020-04-21 20:30:38.034966
137,user_23,laba06s,2020-05-21 14:29:15.709568,NaT
138,user_17,laba06,2020-05-21 15:21:31.567615,2020-04-18 10:56:55.833899


Создадим два фрейма данных: тестовый и контрольный:

у тестового должны быть пользователи, у которых есть значения в first_view_ts

у контрольного должны быть пользователи, у которых отсутствуют значения в first_view_ts

In [21]:
test = datamart[datamart['first_view_ts'].notnull()]
control = datamart[datamart['first_view_ts'].isnull()]

Найдем среднее значения first_view_ts

In [22]:
replase_arg = test['first_view_ts'].mean()
replase_arg

Timestamp('2020-04-27 00:40:05.761783552')

Заменим недостающие значения на средние

In [23]:
control.fillna(replase_arg, inplace=True)

/var/folders/zz/zyxvpxvq6csfxvn_n0001y88000gk2/T/ipykernel_64036/2769073006.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  control.fillna(replase_arg, inplace=True)


Сохраним обе таблицы в базе данных

In [24]:
test.to_sql('test', connect)
control.to_sql('control', connect)

81

Закроем соединение с базой данных

In [ ]:
connect.close()

#ex03

In [ ]:
import pandas as pd
import sqlite3

In [ ]:
from google.colab import drive  # если вы выполняете код из среды Google Colab, нужно подключить свой гугл-диск,
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


Создадим подключение к базе данных с помощью библиотеки sqlite3

In [ ]:
col_names = ['datetime', 'user']
connect = sqlite3.connect('/content/drive/MyDrive/Pyton/checking-logs.sqlite.sqlite copy')

In [ ]:
query = 'PRAGMA table_info(test);'
pd.io.sql.read_sql(query, connect)

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,uid,TEXT,0,None,0
2,2,labname,TEXT,0,None,0
3,3,first_commit_ts,TIMESTAMP,0,None,0
4,4,first_view_ts,TIMESTAMP,0,None,0


Получим первые 10 строк просмотров страниц таблицы, чтобы проверить, как выглядит таблица

In [ ]:
query = "SELECT * FROM test LIMIT 10"
pd.io.sql.read_sql(query, connect)

,index,uid,labname,first_commit_ts,first_view_ts
0,3,user_17,project1,2020-04-18 07:56:45.408648,2020-04-18 10:56:55.833899
1,4,user_30,laba04,2020-04-18 13:36:53.971502,2020-04-17 22:46:26.785035
2,7,user_30,laba04s,2020-04-18 14:51:37.498399,2020-04-17 22:46:26.785035
3,8,user_14,laba04,2020-04-18 15:14:00.312338,2020-04-18 10:53:52.623447
4,11,user_14,laba04s,2020-04-18 22:30:30.247628,2020-04-18 10:53:52.623447
5,18,user_19,laba04,2020-04-20 19:05:01.297780,2020-04-21 20:30:38.034966
6,19,user_25,laba04,2020-04-20 19:16:50.673054,2020-05-09 23:54:54.260791
7,20,user_21,laba04,2020-04-21 17:48:00.487806,2020-04-22 22:40:36.824081
8,21,user_30,project1,2020-04-22 12:36:24.053518,2020-04-17 22:46:26.785035
9,23,user_21,laba04s,2020-04-22 20:09:21.857747,2020-04-22 22:40:36.824081


Посмотрим, что за данные в таблице deadlines

In [ ]:
query = 'PRAGMA table_info(deadlines);'
pd.io.sql.read_sql(query, connect)

,cid,name,type,notnull,dflt_value,pk
0,0,index,INTEGER,0,None,0
1,1,labs,TEXT,0,None,0
2,2,deadlines,INTEGER,0,None,0


In [ ]:
query = "SELECT * FROM deadlines LIMIT 10"
pd.io.sql.read_sql(query, connect)

,index,labs,deadlines
0,0,laba04,1587945599
1,1,laba04s,1587945599
2,2,laba05,1588550399
3,4,laba06,1590364799
4,5,laba06s,1590364799
5,3,project1,1589673599


Видим, что дата в этой таблице указана количества секунд с 1 января 1970 года. Поэтому нам надо будет привести даты к одному виду.

Найдем среди всех пользователей минимальное значение дельты между первой фиксацией пользователя и крайним сроком соответствующей лаборатории, используя только один запрос

CAST позволяет преобразовать данные X в данные другого типа Y

JulianDay количество дней с полудня 24 ноября 4714 года до н. э. https://oracleplsql.ru/julianday-sqlite.html

DATETIME https://bravikov.wordpress.com/2013/08/26/функции-даты-и-временем-sqlite/

unixepoch вычисляет количество секунд с 1 января 1970 года

LEFT JOIN создает левое внешнее соединение. Выбираются все записи первой (левой) таблицы, даже если они не соответствуют записям во второй (правой) таблице.

<> Не равно

In [ ]:
query = '''
SELECT uid,
       MIN(CAST( (JulianDay(DATETIME(deadlines.deadlines, 'unixepoch')) -
                  JulianDay(test.first_commit_ts)
                  ) * 24 AS Integer
                )
           ) AS delta
FROM test
LEFT JOIN deadlines ON test.labname=deadlines.labs
WHERE labname <> 'project1'
'''
df_min = pd.io.sql.read_sql(query, connect)
df_min

,uid,delta
0,user_25,2


Сделаем то же самое, но для максимума

In [ ]:
query = '''
SELECT uid,
       MAX(CAST( (JulianDay(DATETIME(deadlines.deadlines, 'unixepoch')) -
                  JulianDay(test.first_commit_ts)
                  ) * 24 AS Integer
                )
           ) AS delta
FROM test
LEFT JOIN deadlines ON test.labname=deadlines.labs
WHERE labname <> 'project1'
'''
df_max = pd.io.sql.read_sql(query, connect)
df_max

,uid,delta
0,user_30,202


Сделаем то же самое для среднего значения. Фрейм данных не должен включать столбец uid.

In [ ]:
query = '''
SELECT AVG(CAST( (JulianDay(DATETIME(deadlines.deadlines, 'unixepoch')) -
                  JulianDay(test.first_commit_ts)
                  ) * 24 AS Integer
                )
           ) AS delta
FROM test
LEFT JOIN deadlines ON test.labname=deadlines.labs
WHERE labname <> 'project1'
'''
df_avg = pd.io.sql.read_sql(query, connect)
df_avg

,delta
0,89.125


Создадим таблицу со столбцами: uid, avg_diff, pageviews согласно заданию

In [ ]:
query = '''
SELECT test.uid,
       AVG(CAST( (JulianDay(DATETIME(deadlines.deadlines, 'unixepoch')) - 
                  JulianDay(test.first_commit_ts)
                  ) * 24 AS Integer
                )
           ) AS delta, 
       pageviews
FROM test
LEFT JOIN deadlines ON test.labname=deadlines.labs
LEFT JOIN (SELECT uid, count(*) AS pageviews
           FROM pageviews
           GROUP BY uid) AS views ON test.uid=views.uid
WHERE labname <> 'project1'
GROUP BY test.uid
'''
views_diff = pd.io.sql.read_sql(query, connect)
views_diff

,uid,delta,pageviews
0,user_1,64.400000,28
1,user_10,74.800000,89
2,user_14,159.000000,143
3,user_17,61.600000,47
4,user_18,5.666667,3
5,user_19,98.750000,16
6,user_21,95.500000,10
7,user_25,92.600000,179
8,user_28,86.400000,149
9,user_3,105.400000,317


Рассчитаем коэффициент корреляции между количеством просмотров страниц и разницей между первой фиксацией и дедлайном

In [ ]:
views_diff.corr()

,delta,pageviews
delta,1.000000,0.279736
pageviews,0.279736,1.000000


Закроем соединение

In [ ]:
connect.close()

#ex04

In [ ]:
import pandas as pd
import sqlite3

In [ ]:
from google.colab import drive  # если вы выполняете код из среды Google Colab, нужно подключить свой гугл-диск,
drive.mount('/content/drive')

In [ ]:
col_names = ['datetime', 'user']
connect = sqlite3.connect('/content/drive/MyDrive/Pyton/checking-logs.sqlite.sqlite copy')

Создадим два фрейма данных: test_results и control_results со столбцами time и avg_diff 

In [ ]:
query = '''
SELECT time,
       avg(diff) AS avg_diff
FROM (SELECT uid,
             CAST( (JulianDay(datetime(deadlines.deadlines, 'unixepoch')) -
                    JulianDay(test.first_commit_ts)
                    ) * 24 as Integer
                  ) as diff,
             CASE WHEN test.first_commit_ts < test.first_view_ts THEN 'before'
             ELSE 'after' END AS time
       FROM test left join deadlines on test.labname = deadlines.labs
       WHERE labname <> 'project1'
      )
WHERE uid in (SELECT uid
              FROM (SELECT uid,
                           CASE WHEN test.first_commit_ts < test.first_view_ts THEN 'before'
                           ELSE 'after' END as time
                    FROM test
                    LEFT JOIN deadlines ON test.labname=deadlines.labs
                    WHERE labname <> 'project1'
                    )
               GROUP BY uid
               HAVING COUNT(DISTINCT time)=2
               )
GROUP BY time
'''
test_results = pd.io.sql.read_sql(query, connect)
test_results

,time,avg_diff
0,after,104.6000
1,before,60.5625


In [ ]:
query = '''
SELECT time,
       avg(diff) AS avg_diff
FROM (SELECT uid,
             CAST( (JulianDay(datetime(deadlines.deadlines, 'unixepoch')) -
                    JulianDay(control.first_commit_ts)
                    ) * 24 as Integer
                  ) as diff,
             CASE WHEN control.first_commit_ts < control.first_view_ts THEN 'before'
             ELSE 'after'
             END AS time
             FROM control
             LEFT JOIN deadlines ON control.labname=deadlines.labs
             WHERE labname <> 'project1'
      )
WHERE uid IN (SELECT uid
              FROM (SELECT uid,
                           CASE WHEN control.first_commit_ts < control.first_view_ts THEN 'before'
                           ELSE 'after'
                           END AS time
                    FROM control
                    LEFT JOIN deadlines ON control.labname=deadlines.labs
                    WHERE labname <> 'project1'
                    )
              GROUP BY uid
              HAVING COUNT(distinct time)=2
              )
GROUP BY time
'''
control_results = pd.io.sql.read_sql(query, connect)
control_results

,time,avg_diff
0,after,117.636364
1,before,99.464286


В тестовой группе дельта до первого посещения новостной ленты значительно отличается по сравнению с дельтой после.

In [ ]:
percent_test = round((test_results.avg_diff[0] - test_results.avg_diff[1]) / test_results.avg_diff[0] * 100, 0)
print(percent_test, '%')

42.0 %


Эта разница существенно меньше в контрольной группе.

In [ ]:
percent_control = round((control_results.avg_diff[0] - control_results.avg_diff[1]) / control_results.avg_diff[0] * 100, 0)
print(percent_control, '%')

15.0 %


Вывод: гипотеза верна, создание страницы было отличной идеей. 